# 股票数据批量下载 Notebook

本notebook用于批量下载股票历史数据，不做任何分析。

## 功能
- 批量下载沪深300/中证500等指数成分股
- 下载指定股票列表
- 使用download模式（历史纯粹性）
- 自动保存到Repository

## 数据包含
- OHLCV（开高低收成交量）
- 市值数据（Yahoo Finance）
- 财务指标（AkShare）
- 复权因子

## 1. 环境设置

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta

# 添加项目根目录到路径（notebook 在根目录，所以当前目录就是根目录）
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from data_process.data_provider  import HybridDataProvider
from data_process.repository     import DataRepository
from data_process.data_validator import DataValidator
from data_process.data_updater   import DataUpdater, UpdateResult

# 初始化
provider = HybridDataProvider()
repo     = DataRepository("./rawdata/repository")
updater  = DataUpdater(repo, provider)

print("✅ 初始化完成")


C:\Users\mengxinchen\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


✅ 初始化完成


## 2. 设置下载参数

In [2]:
# ========== 配置区域 - 根据需要修改 ==========

# 方式1: 下载指数成分股（二选一）
INDEX_CODE = '000300'  # 000300=沪深300, 000905=中证500, 000016=上证50
LIMIT = None  # 限制下载数量（None=全部下载）

# 方式2: 下载指定股票列表（如果设置了，会覆盖方式1）
SYMBOLS = ['600519', '000001', '000002']  # 例如: ['600519', '000001', '000002'] 或 None

# 日期范围
START_DATE = '2024-01-01'
END_DATE = '2026-02-20'  # 使用T-1日期（昨天）

# 其他选项
SKIP_EXISTING = True  # True=跳过已下载的股票
MODE = 'auto'  # auto=自动选择（推荐）, download=历史下载模式, update=实时更新模式

# ========== 配置结束 ==========

# 智能模式选择（如果设置为 auto）
if MODE == 'auto':
    actual_mode = provider.select_mode(END_DATE)
    print(f"🤖 智能模式选择: {actual_mode}")
else:
    actual_mode = MODE

print("📋 下载配置:")
print(f"  模式: {actual_mode} (设置: {MODE})")
print(f"  日期范围: {START_DATE} ~ {END_DATE}")
print(f"  跳过已存在: {SKIP_EXISTING}")
if SYMBOLS:
    print(f"  股票列表: {SYMBOLS}")
else:
    print(f"  指数: {INDEX_CODE}")
    print(f"  限制数量: {LIMIT if LIMIT else '全部'}")

🤖 智能模式选择: download
📋 下载配置:
  模式: download (设置: auto)
  日期范围: 2024-01-01 ~ 2026-02-20
  跳过已存在: True
  股票列表: ['600519', '000001', '000002']


## 3. 获取股票列表

In [3]:
if SYMBOLS:
    # 使用指定的股票列表
    symbols_to_download = SYMBOLS
    print(f"✅ 使用指定股票列表: {len(symbols_to_download)} 只")
else:
    # 从指数获取成分股
    print(f"正在获取 {INDEX_CODE} 成分股...")
    constituents = provider.get_index_constituents(INDEX_CODE)
    print(f"✅ 获取到 {len(constituents)} 只成分股")
    
    if LIMIT:
        symbols_to_download = constituents.index[:LIMIT].tolist()
        print(f"⚠️ 限制下载数量: {len(symbols_to_download)} 只")
    else:
        symbols_to_download = constituents.index.tolist()

print(f"\n准备下载 {len(symbols_to_download)} 只股票")
print(f"前10只: {', '.join(symbols_to_download[:10])}")

✅ 使用指定股票列表: 3 只

准备下载 3 只股票
前10只: 600519, 000001, 000002


## 4. 开始批量下载

In [4]:
print("="*80)
print("开始批量下载...")
print("="*80)
print()

# 结果字典: key = "symbol_dtype" → UpdateResult
all_results = {}

DATA_TYPES = ['ohlcv', 'adjustment', 'market_value', 'financial']

for i, symbol in enumerate(symbols_to_download, 1):
    print(f"[{i}/{len(symbols_to_download)}] 处理 {symbol}...")
    for dtype in DATA_TYPES:
        result = updater.update_symbol(
            symbol,
            start_date=START_DATE,
            end_date=END_DATE,
            data_type=dtype
        )
        all_results[f"{symbol}_{dtype}"] = result
        print(result)   # UpdateResult.__str__ 已有格式化输出（✅/❌ + 统计信息）
    print()

print("="*80)
print(f"✅ 批量下载完成，共处理 {len(symbols_to_download)} 只股票")
print("="*80)


开始批量下载...

[1/3] 处理 600519...
✅ [ohlcv] 600519
   新增 515 行  重叠 0 行  变化 0 格
✅ [adjustment] 600519
   新增 515 行  重叠 0 行  变化 0 格
✅ [market_value] 600519
   新增 515 行  重叠 0 行  变化 0 格
✅ [financial] 600519
   新增 100 行  重叠 0 行  变化 0 格

[2/3] 处理 000001...
✅ [ohlcv] 000001
   新增 515 行  重叠 0 行  变化 0 格
✅ [adjustment] 000001
   新增 515 行  重叠 0 行  变化 0 格
✅ [market_value] 000001
   新增 515 行  重叠 0 行  变化 0 格
✅ [financial] 000001
   新增 120 行  重叠 0 行  变化 0 格

[3/3] 处理 000002...
✅ [ohlcv] 000002
   新增 515 行  重叠 0 行  变化 0 格
✅ [adjustment] 000002
   新增 515 行  重叠 0 行  变化 0 格
✅ [market_value] 000002
   新增 515 行  重叠 0 行  变化 0 格
✅ [financial] 000002
   新增 114 行  重叠 0 行  变化 0 格

✅ 批量下载完成，共处理 3 只股票


## 5. 下载结果汇总

In [5]:
total        = len(all_results)
success      = sum(1 for r in all_results.values() if r.success)
failed       = total - success
new_rows_total = sum(r.new_rows for r in all_results.values() if r.success)
has_warnings = sum(1 for r in all_results.values() if r.success and r.warnings)

print("📊 下载结果统计")
print("="*80)
print(f"  总任务数 (股票 × 数据类型): {total}")
print(f"  ✅ 成功: {success}")
print(f"  ❌ 失败: {failed}")
print(f"  ⚠️  有警告: {has_warnings}")
print(f"\n  📈 共新增数据行: {new_rows_total}")

# 打印有警告或失败的明细
flagged = {k: r for k, r in all_results.items() if not r.success or r.warnings}
if flagged:
    print(f"\n⚠️  需关注的任务 ({len(flagged)} 个):")
    print("-"*80)
    for key, result in flagged.items():
        print(result)
else:
    print("\n✅ 所有任务均正常完成，无警告。")


📊 下载结果统计
  总任务数 (股票 × 数据类型): 12
  ✅ 成功: 12
  ❌ 失败: 0
  ⚠️  有警告: 0

  📈 共新增数据行: 4969

✅ 所有任务均正常完成，无警告。


## 6. 失败股票列表（如有）

In [6]:
failed_items = {k: r for k, r in all_results.items() if not r.success}

if failed_items:
    print(f"⚠️ 以下 {len(failed_items)} 个任务失败:")
    print("="*80)
    for key, r in failed_items.items():
        print(f"  {key}: {r.error}")
    print("\n💡 建议: 可重新运行 cell-8，或调整 START_DATE/END_DATE 后重试")
else:
    print("✅ 所有任务成功！")


✅ 所有任务成功！


## 7. 导出结果到Excel（可选）

In [7]:
# 导出结果
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f'data/exports/download_report_{timestamp}.xlsx'

# 创建目录
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# 保存
results_df.to_excel(output_file, index=False)
print(f"✅ 结果已导出到: {output_file}")

NameError: name 'results_df' is not defined

## 8. 数据存储位置

In [ ]:
print("📂 数据存储位置:")
print("="*80)
print(f"\n项目根目录: {project_root}")
print(f"\n✅ DataRepository标准目录结构:")
print("data/")
print("└── repository/")
print("    ├── market/")
print("    │   └── daily/")
print("    │       ├── ohlcv/           ← OHLCV数据")
print("    │       │   └── {symbol}/")
print("    │       │       ├── data.csv")
print("    │       │       └── metadata.json")
print("    │       └── adjustment/      ← 复权因子")
print("    │           └── {symbol}/")
print("    │               ├── data.csv")
print("    │               └── metadata.json")
print("    └── fundamental/")
print("        ├── market_value/        ← 市值数据")
print("        │   └── {symbol}/")
print("        │       ├── data.csv")
print("        │       └── metadata.json")
print("        └── financial_indicators/ ← 财务数据")
print("            └── {symbol}/")
print("                ├── data.csv")
print("                └── metadata.json")

# 检查数据目录
repo_dir = os.path.join(project_root, 'rawdata', 'repository', 'market', 'daily', 'ohlcv')
if os.path.exists(repo_dir):
    stock_count = len([d for d in os.listdir(repo_dir) if os.path.isdir(os.path.join(repo_dir, d))])
    print(f"\n✅ 当前已下载 {stock_count} 只股票的数据")
    
    # 验证数据格式
    if stock_count > 0:
        # 检查第一个股票的格式
        sample_symbol = [d for d in os.listdir(repo_dir) if os.path.isdir(os.path.join(repo_dir, d))][0]
        sample_dir = os.path.join(repo_dir, sample_symbol)
        has_data_csv = os.path.exists(os.path.join(sample_dir, 'data.csv'))
        has_metadata = os.path.exists(os.path.join(sample_dir, 'metadata.json'))
        
        print(f"\n🔍 格式验证（示例: {sample_symbol}）:")
        print(f"  - data.csv存在: {'✅' if has_data_csv else '❌'}")
        print(f"  - metadata.json存在: {'✅' if has_metadata else '❌'}")
        
        if has_data_csv and has_metadata:
            print(f"  - 格式符合Repository标准: ✅")
else:
    print(f"\n⚠️ 数据目录不存在: {repo_dir}")
    print("提示: 首次运行时会自动创建目录结构")

📂 数据存储位置:

项目根目录: d:\2026_claude\StockFrame

✅ DataRepository标准目录结构:
data/
└── repository/
    ├── market/
    │   └── daily/
    │       ├── ohlcv/           ← OHLCV数据
    │       │   └── {symbol}/
    │       │       ├── data.csv
    │       │       └── metadata.json
    │       └── adjustment/      ← 复权因子
    │           └── {symbol}/
    │               ├── data.csv
    │               └── metadata.json
    └── fundamental/
        ├── market_value/        ← 市值数据
        │   └── {symbol}/
        │       ├── data.csv
        │       └── metadata.json
        └── financial_indicators/ ← 财务数据
            └── {symbol}/
                ├── data.csv
                └── metadata.json

✅ 当前已下载 3 只股票的数据

🔍 格式验证（示例: 000001）:
  - data.csv存在: ✅
  - metadata.json存在: ✅
  - 格式符合Repository标准: ✅


---

## 总结

### ✅ 完成的工作
- 批量下载股票历史数据
- 数据自动保存到Repository
- 生成下载结果报告

### 📊 数据说明
- **模式**: auto（智能选择模式）
- **覆盖率**: 根据 end_date 自动选择 download 或 update
- **数据源**: Yahoo Finance（市值） + AkShare（OHLCV、财务）

### 💡 下一步
1. 使用 `data_analysis_example.ipynb` 进行数据分析
2. 每日使用update模式更新最新数据
3. 开始因子挖掘和回测

### 📚 相关文档
- [智能模式选择](docs/AUTO_MODE_SELECTION.md)
- [智能模式选择示例](example_auto_mode_selection.py)

---

**状态**: ✅ 下载完成

---

## 9. 日常增量更新（使用 DataUpdater）

**与上面 cell-8 的区别**：仅下载范围不同，流程完全一致。

| | cell-8（批量下载） | cell-19（日常增量） |
|---|---|---|
| 典型用途 | 初次下载 / 补历史数据 | 每日追加最新交易日 |
| start_date | 用户指定（如 `2024-01-01`） | 近 30 天（减少下载量） |
| 流程 | provider → validator → compare_and_merge → save | 相同 |

`DataUpdater.update_all()` 会：
1. 对 repo 中所有已有股票，按 `start_date ~ end_date` 全量下载
2. 通过 `DataValidator` 验证字段/逻辑正确性
3. 通过 `compare_and_merge` 识别新增行和变化行
4. 检查日期连贯性、hfq_factor 单调性、列结构一致性
5. 纯覆盖写入（新数据已是合并后的完整数据）
6. 返回结构化 `UpdateResult` 用于审计与监控

> 💡 `update_all` 默认只更新 repo 中 **已有股票**（从 `ohlcv` 目录扫描）。
> 新股票请先在 cell-8 中下载。


In [ ]:
# ========== 日常增量更新 - 配置区域 ==========

UPDATE_END_DATE   = datetime.today().strftime('%Y-%m-%d')  # 截止今天
# 只下载近30天数据（足以覆盖所有新增交易日，同时大幅减少下载量）
UPDATE_START_DATE = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')
UPDATE_SYMBOLS    = None   # None = 更新 repo 中所有已有股票
UPDATE_DTYPES     = None   # None = 全部类型（ohlcv / adjustment / market_value / financial）
SKIP_ON_ERROR     = True   # 遇到单只股票失败时继续（不中断整批）

# ========== 执行增量更新 ==========

print("="*80)
print(f"日常增量更新  {UPDATE_START_DATE} ~ {UPDATE_END_DATE}")
print("="*80)

update_results = updater.update_all(
    start_date    = UPDATE_START_DATE,
    end_date      = UPDATE_END_DATE,
    data_types    = UPDATE_DTYPES,
    symbols       = UPDATE_SYMBOLS,
    skip_on_error = SKIP_ON_ERROR,
)

# ========== 打印摘要 ==========

total   = len(update_results)
success = sum(1 for r in update_results.values() if r.success)
skipped = sum(1 for r in update_results.values() if r.success and r.new_rows == 0 and r.overlap_rows == 0)
warnings_count = sum(1 for r in update_results.values() if r.success and r.warnings)
failed  = total - success

print(f"\n📊 更新结果统计")
print("="*80)
print(f"  总任务数: {total}")
print(f"  ✅ 成功:  {success}  （含 {skipped} 个无新数据）")
print(f"  ⚠️  有警告: {warnings_count}")
print(f"  ❌ 失败:  {failed}")

# 新增行数汇总
new_rows_total = sum(r.new_rows for r in update_results.values() if r.success)
print(f"\n  📈 共新增数据行: {new_rows_total}")

# 打印有警告或失败的明细
flagged = {k: r for k, r in update_results.items() if not r.success or r.warnings}
if flagged:
    print(f"\n⚠️  需关注的任务 ({len(flagged)} 个):")
    print("-"*80)
    for key, result in flagged.items():
        print(result)
else:
    print("\n✅ 所有任务均正常完成，无警告。")
